# 3. LangChain의 기초

In [3]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## (1) LangChain 개요
### LangChain 설치

In [1]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.15
    Uninstalling langchain-core-0.3.15:
      Successfully uninstalled langchain-core-0.3.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.7 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.3.0 which is incompatible.
langchain-text-splitters 0.3.2 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.3.0 which is incompatible.


### LangSmith 설정

In [4]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-test"

## (2) LLM/Chat model
### LLM

In [6]:
from langchain_openai import OpenAI

model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
ai_message = model.invoke("안녕하세요.")
print(ai_message)

 저는 김민지입니다. 만나서 반가워요. 저는 대학교에서 경영학을 전공하고 있어요. 취미는 음악 감상이고, 특히 팝 음악을 좋아해요. 또한 여행을 좋아해서 여러 나라를 다녀보는 것이 꿈이에요. 앞으로 잘 부탁드립니다!


### Chat model

In [7]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕하세요! 저는 홍길동이라고 합니다!"),
    AIMessage(content="안녕하세요, 홍길동님! 무엇을 도와드릴까요?"),
    HumanMessage(content="제 이름을 아시나요?"),
]

ai_message = model.invoke(messages)
print(ai_message.content)

네, 홍길동님이라고 말씀하셨습니다! 어떻게 도와드릴까요?


### 스트리밍

In [8]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕하세요!"),
]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

안녕하세요! 어떻게 도와드릴까요?

## (3) Prompt template

### PromptTemplate

In [9]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""다음 요리 레시피를 생각해 봅시다.

요리명: {dish}""")

prompt_value = prompt.invoke({"dish": "카레"})
print(prompt_value.text)

다음 요리 레시피를 생각해 봅시다.

요리명: 카레


### ChatPromptTemplate

In [10]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리 레시피를 생각해 주세요."),
        ("human", "{dish}"),
    ]
)

prompt_value = prompt.invoke({"dish": "카레"})
print(prompt_value)

messages=[SystemMessage(content='사용자가 입력한 요리 레시피를 생각해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='카레', additional_kwargs={}, response_metadata={})]


### MesaagesPlaceholder

In [11]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="안녕하세요. 저는 홍길동이라고 합니다."),
            AIMessage("안녕하세요, 홍길동님! 무엇을 도와드릴까요?"),
        ],
        "input": "제 이름을 아시나요?",
    }
)
print(prompt_value)

messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요. 저는 홍길동이라고 합니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 홍길동님! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='제 이름을 아시나요?', additional_kwargs={}, response_metadata={})]


### LangSmith의 Prompts

In [15]:
from langsmith import Client

client = Client()

prompt = client.pull_prompt("zer0/recipe")
prompt_value = prompt.invoke({"dish": "카레"})
print(prompt_value)

LangSmithNotFoundError: Resource not found for /commits/zer0/recipe/latest. HTTPError('404 Client Error: Not Found for url: https://api.smith.langchain.com/commits/zer0/recipe/latest', '{"detail":"Repo zer0/recipe not found"}')

## (4) Output parser

### PydanticOutputParser를 사용하여 Python객체로 변환하는 방법

In [16]:
from pydantic import BaseModel, Field

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

In [17]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [18]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [19]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "사용자가 입력한 요리 레시피를 생각해보세요.\n\n"
            "{format_instructions}",
        ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)

In [20]:
prompt_value = prompt_with_format_instructions.invoke({"dish": "카레"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)

=== role: system ===
사용자가 입력한 요리 레시피를 생각해보세요.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: user ===
카레


In [21]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

ai_message = model.invoke(prompt_value)
print(ai_message.content)

```json
{
  "ingredients": [
    "닭고기 500g",
    "양파 1개",
    "감자 2개",
    "당근 1개",
    "카레 가루 3큰술",
    "코코넛 밀크 400ml",
    "식용유 2큰술",
    "소금 약간",
    "후추 약간"
  ],
  "steps": [
    "닭고기를 한입 크기로 썰고, 소금과 후추로 간을 한다.",
    "양파는 다지고, 감자와 당근은 깍둑썰기로 준비한다.",
    "팬에 식용유를 두르고 양파를 볶아 투명해질 때까지 익힌다.",
    "닭고기를 넣고 겉면이 노릇해질 때까지 볶는다.",
    "감자와 당근을 추가하고 잘 섞은 후, 카레 가루를 넣고 볶는다.",
    "코코넛 밀크를 붓고 끓인다.",
    "중불로 줄이고 20분간 끓여서 재료가 부드러워질 때까지 조리한다.",
    "소금으로 간을 맞추고, 그릇에 담아낸다."
  ]
}
```


In [22]:
recipe = output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기 500g', '양파 1개', '감자 2개', '당근 1개', '카레 가루 3큰술', '코코넛 밀크 400ml', '식용유 2큰술', '소금 약간', '후추 약간'] steps=['닭고기를 한입 크기로 썰고, 소금과 후추로 간을 한다.', '양파는 다지고, 감자와 당근은 깍둑썰기로 준비한다.', '팬에 식용유를 두르고 양파를 볶아 투명해질 때까지 익힌다.', '닭고기를 넣고 겉면이 노릇해질 때까지 볶는다.', '감자와 당근을 추가하고 잘 섞은 후, 카레 가루를 넣고 볶는다.', '코코넛 밀크를 붓고 끓인다.', '중불로 줄이고 20분간 끓여서 재료가 부드러워질 때까지 조리한다.', '소금으로 간을 맞추고, 그릇에 담아낸다.']


### StrOutputParser

In [23]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="안녕하세요. 저는 AI 어시스턴트입니다.")
ai_message = output_parser.invoke(ai_message)
print(type(ai_message))
print(ai_message)

<class 'str'>
안녕하세요. 저는 AI 어시스턴트입니다.


## (5) Chain-LangChain Expression Language(LCEL) 개요

### prompt와 model의 이해

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리 레시피를 생각해보세요."),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [25]:
chain = prompt | model

In [26]:
ai_message = chain.invoke({"dish": "카래"})
print(ai_message.content)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 기본적인 카레 레시피를 소개해드릴게요.

### 재료
- 고기 (닭고기, 소고기, 돼지고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 코코넛 밀크 또는 물 2컵
- 식용유 2 큰술
- 소금, 후추 약간
- 선택 재료: 마늘, 생강, 피망, 버섯 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파, 감자, 당근은 깍둑썰기 합니다. 마늘과 생강은 다져줍니다.
   
2. **양파 볶기**: 팬에 식용유를 두르고 중불에서 양파를 볶아 투명해질 때까지 볶습니다.

3. **고기 추가**: 양파가 볶아지면 고기를 넣고 겉면이 익을 때까지 볶습니다.

4. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다.

5. **카레 가루 추가**: 카레 가루를 넣고 잘 섞어줍니다. 향이 올라올 때까지 1-2분 더 볶습니다.

6. **액체 추가**: 코코넛 밀크 또는 물을 넣고 끓입니다. 끓기 시작하면 불을 줄이고 뚜껑을 덮고 20-30분 정도 끓입니다. 중간에 간을 보고 소금과 후추로 간을 맞춥니다.

7. **완성**: 모든 재료가 부드러워지면 불을 끄고, 밥과 함께 서빙합니다.

### 팁
- 카레는 시간이 지날수록 맛이 깊어지므로, 미리 만들어 두면 더 맛있습니다.
- 다양한 채소나 해산물을 추가하여 변화를 줄 수 있습니다.

맛있게 만들어 보세요!


### StrOutputParser를 체인에 추가

In [27]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish": "카레"})
print(output)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 양고기 등) 300g
- 감자 1개
- 당근 1개
- 양파 1개
- 마늘 2쪽
- 생강 1 작은 조각
- 카레 가루 2-3 큰술
- 코코넛 밀크 또는 물 400ml
- 식용유 2 큰술
- 소금, 후추 약간
- 선택 재료: 피망, 버섯, 완두콩 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 감자와 당근은 깍둑썰기, 양파는 다지고, 마늘과 생강은 다져줍니다.

2. **양파 볶기**: 팬에 식용유를 두르고 중불에서 양파를 볶아 투명해질 때까지 볶습니다.

3. **마늘과 생강 추가**: 다진 마늘과 생강을 넣고 향이 날 때까지 볶습니다.

4. **고기 추가**: 고기를 넣고 겉면이 익을 때까지 볶습니다.

5. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다.

6. **카레 가루 추가**: 카레 가루를 넣고 잘 섞어줍니다. 이때 향신료의 향이 나기 시작합니다.

7. **액체 추가**: 코코넛 밀크 또는 물을 부어주고, 끓어오르면 불을 줄여서 20-30분 정도 끓입니다. 중간에 간을 보고 소금과 후추로 간을 맞춥니다.

8. **완성**: 모든 재료가 부드럽게 익으면 불을 끄고, 밥과 함께 서빙합니다.

### 팁
- 카레는 시간이 지날수록 맛이 깊어지므로, 다음 날 먹으면 더욱 맛있습니다.
- 다양한 채소나 해산물을 추가하여 변화를 줄 수 있습니다.

맛있게 만들어 보세요!


### PydanticOutputParser를 사용하는 체인

In [28]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리 레시피를 생각해보세요.\n\n{format_instructions}"),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=output_parser.get_format_instructions()
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind(
    response_format={"type": "json_object"}
)

In [30]:
chain = prompt_with_format_instructions | model | output_parser

In [31]:
recipe = chain.invoke({"dish": "카레"})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기 500g', '양파 1개', '감자 2개', '당근 1개', '카레 가루 3큰술', '식용유 2큰술', '소금 약간', '후추 약간', '물 4컵'] steps=['양파를 잘게 썰고, 감자와 당근은 큐브 모양으로 자릅니다.', '냄비에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶습니다.', '닭고기를 넣고 겉면이 노릇해질 때까지 볶습니다.', '감자와 당근을 넣고 함께 볶습니다.', '카레 가루를 넣고 잘 섞은 후 물을 부어 끓입니다.', '끓기 시작하면 중약불로 줄이고 20분간 끓입니다.', '소금과 후추로 간을 맞춘 후 불을 끄고 5분 정도 둡니다.', '밥과 함께 서빙합니다.']


## ⑹ LangChain의 RAG관련 컴포넌트

### RAG(검색증강생성, Retrieval-Augmented Generation)
### Document loader

In [32]:
!pip install langchain-community==0.3.0 GitPython==3.1.43


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.0
    Uninstalling langchain-core-0.3.0:
      Successfully uninstalled langchain-core-0.3.0


In [33]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))

371


### Document transformer

In [34]:
!pip install langchain-text-splitters==0.3.0

  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.2
    Uninstalling langchain-text-splitters-0.3.2:
      Successfully uninstalled langchain-text-splitters-0.3.2


In [35]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(raw_docs)
print(len(docs))

1270


### Embedding model

In [36]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [37]:
query = "AWS S3에서 데이터를 불러올 수 있는 Document loader가 있나요?"

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

1536
[0.01985582523047924, 0.02229463681578636, 0.028337715193629265, -0.01978028565645218, 0.04907841980457306, 0.016888242214918137, -0.01323002204298973, 0.009534033015370369, 0.027841318398714066, -0.024344967678189278, -0.0006370185292325914, -0.012086153961718082, -0.006177966948598623, -0.018323473632335663, -0.01399619784206152, 0.06509257107973099, -0.0022472692653536797, 0.00711140688508749, -0.03774764761328697, 0.004615939687937498, 0.017622044309973717, 0.01573358289897442, -0.03578364849090576, 0.017125649377703667, 0.03515775874257088, -0.030280131846666336, -0.015917032957077026, 0.025812571868300438, -0.008999868296086788, -0.09435833245515823, 0.0013650879263877869, -0.0636681318283081, -0.02576940692961216, 0.04251736402511597, -0.018755121156573296, 0.044028133153915405, 0.04519358277320862, 0.012409890070557594, 5.574755505222129e-06, -0.03066861629486084, -0.003485560417175293, -0.02186298929154873, 0.019089648500084877, 0.004178895615041256, 0.030366461724042892,

### Vecor Store

In [38]:
!pip install langchain-chroma==0.1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.2/159.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 4.7 MB/s eta 0:00:

In [39]:
from langchain_chroma import Chroma

db = Chroma.from_documents(docs, embeddings)

In [40]:
retriever = db.as_retriever()

In [41]:
query = "AWS S3에서 데이터를 불러올 수 있는 Document loader가 있나요?"

context_docs = retriever.invoke(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)

len = 4
metadata = {'file_name': 'aws.mdx', 'file_path': 'docs/docs/integrations/providers/aws.mdx', 'file_type': '.mdx', 'source': 'docs/docs/integrations/providers/aws.mdx'}
## Document loaders

### AWS S3 Directory and File

>[Amazon Simple Storage Service (Amazon S3)](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
> is an object storage service.
>[AWS S3 Directory](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
>[AWS S3 Buckets](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingBucket.html)

See a [usage example for S3DirectoryLoader](/docs/integrations/document_loaders/aws_s3_directory).

See a [usage example for S3FileLoader](/docs/integrations/document_loaders/aws_s3_file).

```python
from langchain_community.document_loaders import S3DirectoryLoader, S3FileLoader
```

### Amazon Textract

>[Amazon Textract](https://docs.aws.amazon.com/managedservices/latest/userguide/textract.html) is a machine 
> learning (ML) serv

### LCEL을 이용한 RAG의 Chain구현

In [42]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
아래 문맥을 고려하여 질문에 답해 주세요.

문맥: """
{context}
"""

질문: {question}
''')

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [43]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke(query)
print(output)

네, AWS S3에서 데이터를 불러올 수 있는 Document loader가 있습니다. `S3DirectoryLoader`와 `S3FileLoader`라는 두 가지 Document loader가 제공됩니다. 이들 loader를 사용하여 AWS S3에서 데이터를 불러올 수 있습니다. 사용 예제는 [여기](https://docs/integrations/document_loaders/aws_s3_directory)와 [여기](https://docs/integrations/document_loaders/aws_s3_file)에서 확인할 수 있습니다.
